In [38]:
import pandas as pd
from gensim.utils import simple_preprocess
import gensim.downloader as api
from gensim.models import KeyedVectors
import contractions
import numpy as np
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import utils.utils as utl

torch.random.seed = 2124
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

dataset_name = 'cstm_ds'
tk_model = 'gl_300'

context_win = [4,5,6,7]
No_CNN_filter = 150
lr = 0.0001
max_len = 150
batch_size = 300
regularization=0.01

num_epochs = 10

def preprocess_text(data):
    dataset = []
    for point in data:
        if type(point[1])==str:
            dataset.append((simple_preprocess(contractions.fix(point[1])),point[0]))
    return dataset

out_dim=5
data = np.array(pd.read_excel('./data/Aug24-Assignmen1-Dataset1-test.xlsx')).tolist()
np.random.shuffle(data)
test_dataset= preprocess_text(data)
weight = 0

print(f"train_dataset: {len(test_dataset)}")

import gensim
# print(gensim.downloader.info()['models'].keys())

tk_dict = {'ft_300':'fasttext-wiki-news-subwords-300', 
           'gl_300':'glove-wiki-gigaword-300',
           'gl_100':'glove-wiki-gigaword-100',
           'w2v_300':'word2vec-google-news-300'}

# load model
if os.path.exists(f"./wv_models/{tk_dict[tk_model]}.kv"):
    word_vector = KeyedVectors.load(f"./wv_models/{tk_dict[tk_model]}.kv")
    vec_dim = len(word_vector['apple'])
    print(tk_model,vec_dim)
else:
    # load word vector model
    word_vector = api.load(tk_dict[tk_model])
    # Save the model for future use
    word_vector.save(f"./wv_models/{tk_dict[tk_model]}.kv")
    vec_dim = len(word_vector['apple'])
    print(tk_model,vec_dim)

import models.CNN as cnn

Cnn_model = cnn.CNN_Classifier_network(vec_dim, No_CNN_filter, out_dim, weight=weight,context_win=context_win).to(device)
# load model
e=38
ac=0.1638
type_='best'

if os.path.exists(f"model_CNN_{type_}({utl.checkpoint_name(e,tk_model,(ac),context_win)})_f_{No_CNN_filter}_{dataset_name}.pth"):
    print('loading pretrained model...')
    Cnn_model.load(f'{type_}({utl.checkpoint_name(e,tk_model,(ac),context_win)})_f_{No_CNN_filter}_{dataset_name}')
    print('done')
else:
    print('No pretrained model')

print('_______________________')
print('Validation Dataset: ')

test_loss, test_acc = cnn.evaluate(val_dataset, batch_size, word_vector, vec_dim, tk_model, Cnn_model, max(context_win), max_len, test=True)

if 'test_dataset' in locals() or 'test_dataset' in globals():
    print()
    print('_______________________')
    print('Test Dataset: ')
    test_loss, test_acc = cnn.evaluate(test_dataset, word_vector, vec_dim, tk_model, Cnn_model, max(context_win), test=True)


cuda
train_dataset: 3998
val_dataset: 802
gl_300 300
tensor([ 4.1216, 12.8968,  7.5434,  1.8467,  0.2496], device='cuda:0')
loading pretrained model...
done
_______________________
Validation Dataset: 
_____________________
class label: 0
.	F	T
N	27	739
P	24	12
---------------------
Recall:		 0.3077
Precision:	 0.3333
F1-Score: 	 0.3200
Class Accur.: 	 93.6409
_____________________
class label: 1
.	F	T
N	13	789
P	0	0
---------------------
Recall:		 0.0000
Precision:	 0.0000
F1-Score: 	 0.0000
Class Accur.: 	 98.3791
_____________________
class label: 2
.	F	T
N	21	778
P	2	1
---------------------
Recall:		 0.0455
Precision:	 0.3333
F1-Score: 	 0.0800
Class Accur.: 	 97.1322
_____________________
class label: 3
.	F	T
N	44	508
P	207	43
---------------------
Recall:		 0.4943
Precision:	 0.1720
F1-Score: 	 0.2552
Class Accur.: 	 68.7032
_____________________
class label: 4
.	F	T
N	190	99
P	62	451
---------------------
Recall:		 0.7036
Precision:	 0.8791
F1-Score: 	 0.7816
Class Accur.: 	 68.

In [4]:
import torch
import torch.nn.functional as F

# Assume you have a tensor of shape (812, 1) with integers 0 to 4
input_tensor = torch.randint(0, 5, (8, 1))

# Convert the tensor to a one-hot encoded tensor
# The second argument (num_classes=5) specifies the number of classes (0 to 4)
one_hot_tensor = F.one_hot(input_tensor.squeeze(), num_classes=5)
print(one_hot_tensor)


torch.Size([8, 1])
tensor([[0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0]])


# run

In [10]:
import pandas as pd
import gensim.downloader as api
from gensim.models import KeyedVectors
from transformers import BertTokenizer
from datasets import load_dataset
from gensim.utils import simple_preprocess
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random
import os
import contractions

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import utils.utils as utl

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/adityab/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
import logging

# Set up logging
log_filename = './app.log'
logging.basicConfig(
    filename=log_filename,            # Log output file
    level=logging.DEBUG,              # Logging level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  # Log format
    datefmt='%Y-%m-%d %H:%M:%S'       # Date format
)

# Create a logger object
logger = logging.getLogger(__name__)

# Log some messages
logger.debug('This is a debug message')
logger.info('This is an info message')
logger.warning('This is a warning message')
logger.error('This is an error message')
logger.critical('This is a critical message')

# Additional code...


## **Data Processing**

In [12]:
# dataset = load_dataset('glue', 'sst2')

In [14]:
# import contractions

# text = dataset['train'][90]['sentence']
 
# # creating an empty list
# expanded_words = []    
# for word in text.split():
#   # using contractions.fix to expand the shortened words
#   expanded_words.append(contractions.fix(word))   
   
# expanded_text = ' '.join(expanded_words)
# print('Original text: ' + text)
# print('Expanded_text: ' + expanded_text)

In [15]:
# a = "I don't fell so well"
# tokenizer.tokenize()

In [16]:
# np.random.shuffle(dataset)

In [17]:
# class sstDataset(Dataset):
#     def __init__(self, data, tokentizer, wordvector, batch_size):
#         self.data = data
#         self.batch_size = batch_size
#         self.tokentizer = tokentizer
#         self.wordvector = wordvector

#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, idx):
#         X = self.data[idx*self.batch_size : idx*self.batch_size + self.batch_size]['sentence']
#         Y = self.data[idx*self.batch_size : idx*self.batch_size + self.batch_size]['label']
        

In [18]:
# select dataset 
# 'sst2', 'cstm_ds'
dataset_name = 'cstm_ds'

In [19]:
# data set
if dataset_name =='sst2':
    out_dim=2
    dataset = load_dataset('glue', 'sst2')
    utl.load_SST2(dataset)

    # Preprocess the text data (e.g., tokenizing, cleaning)
    def preprocess_text(text):
        sentences = [(simple_preprocess(text[0][i]),text[1][i]) for i in range(len(text[0]))]
        return sentences  # Tokenizes the text into lowercase words

    # print(dataset['train'][0])
    train_dataset = preprocess_text(dataset['train'])
    val_dataset = preprocess_text(dataset['val'])
    test_dataset = preprocess_text(dataset['test'])
    print("train_dataset: ",len(train_dataset))
    print("val_dataset: ",len(val_dataset))
    print("test_dataset: ",len(test_dataset))

elif dataset_name=='cstm_ds':
    def preprocess_text(data):
        dataset = []
        for point in data:
            if type(point[1])==str:
                dataset.append((simple_preprocess(contractions.fix(point[1])),point[0]))
            else:
                dataset.append(([' '],point[0]))

        return dataset
    out_dim=5
    data = np.array(pd.read_excel('./data/Aug24-Assignmen1-Dataset1.xlsx')).tolist()
    np.random.shuffle(data)
    train_dataset= preprocess_text(data)
    data = np.array(pd.read_excel('./data/Aug24-Assignment1-Validation-Dataset1.xlsx')).tolist()
    np.random.shuffle(data)
    val_dataset = preprocess_text(data)
    upS = utl.inbalanceHandler(train_dataset)
    # train_dataset = upS.upSample(train_dataset)
    print(f"train_dataset: {len(train_dataset)}")
    print(f"val_dataset: {len(val_dataset)}")


train_dataset: 3999
val_dataset: 802


In [13]:
# Open the .log file and read its contents
with open('model1.log', 'r') as file:
    log_content = file.read()
split_text = log_content.split('\n')
for i in (split_text[4:-1]):
    print(i.split('-')[-1])


 model information
 word vector model: gl_300
 Dataset name: cstm_ds
 CNN info...
 vector dimension: 300
 No. of CNN filters: 200
 context window sizes: [4, 5, 6, 7]
 dropout regularization: None
 Linear layer info...
 hidden units: [1000]
 No of output layers: 5
 best model, Epoch [12/500], val accuracy = 44.5137| F1 scores: [0.2585 0.1053 0.1579 0.1995 0.6006], minority mean=0.1803
 last model, Epoch [96/500],: train loss = 0.0722, train acc: 99.9500%| F1 scores: [1.     1.     1.     0.9977 0.9997]


In [23]:
(3999/300)//5

2.0

In [12]:
import re

text = 'I just received my card, it is the class 10 64GB with the **correct** serial#SDSDQUA-064G-A11A, &#34;Fool-proof&#34; . (15 MB/s vs 10 write; 45 MB/s vs 22 MB/s read) on the same card reader.'

# Remove the serial code
text_without_serial = re.sub(r'serial#[\w-]+', '', text)
text_without_html_entity = text_without_serial.replace('&#34;', '')
text_without_html_entity = text_without_html_entity.replace('-', ' ')
text_without_html_entity = text_without_html_entity.replace('*', ' ')
final_text = text_without_html_entity.replace("MB/s", "megabyte per second")

print(final_text)


I just received my card, it is the class 10 64GB with the   correct   , Fool proof . (15 megabyte per second vs 10 write; 45 megabyte per second vs 22 megabyte per second read) on the same card reader.


In [13]:
data[0][1]

'My main intent in getting this particular book was for my wife, who has a MacBook (we both have iMacs - for which much information is still applicable). Keep in mind that this book is written with Snow Leopard as the intended OS, but there is not much difference from Leopard, if that is what you have.  Her impressions:  *Has easy step-by-step instructions with numbered illustrations  *Helpful hints/tips/shortcuts  *Easy to read for the lay person (not heavy with tech jargon)  *Covers wide variety of topics  *High quality, portable reference  My impression is that one cannot go wrong with purchasing this as a guide to the Mac/MacBook, especially if one is highly accustomed to Windows or a much earlier version of Mac OS, though all Macs come with helpful guides within the system that can be found in Spotlight or Help. The Apple website has many sources of helpful information, also, such as tutorials.'

In [14]:


# Example sentence with contractions
sentence = data[0][1]

# Expanding the contractions
expanded_sentence = contractions.fix(sentence)

print(expanded_sentence)


My main intent in getting this particular book was for my wife, who has a MacBook (we both have iMacs - for which much information is still applicable). Keep in mind that this book is written with Snow Leopard as the intended OS, but there is not much difference from Leopard, if that is what you have.  Her impressions:  *Has easy step-by-step instructions with numbered illustrations  *Helpful hints/tips/shortcuts  *Easy to read for the lay person (not heavy with tech jargon)  *Covers wide variety of topics  *High quality, portable reference  My impression is that one cannot go wrong with purchasing this as a guide to the Mac/MacBook, especially if one is highly accustomed to Windows or a much earlier version of Mac OS, though all Macs come with helpful guides within the system that can be found in Spotlight or Help. The Apple website has many sources of helpful information, also, such as tutorials.


In [15]:
a = np.zeros(5)
for d in val_dataset:
    # print(d[0])
    if type(d[0])!= list:
        print(d[0])
    else:
        if len(d[0])<=1:
            print(d[0], d[1])
# print(a)

## **Word vector**


In [16]:
import gensim
print(gensim.downloader.info()['models'].keys())

tk_dict = {'ft_300':'fasttext-wiki-news-subwords-300', 
           'gl_300':'glove-wiki-gigaword-300',
           'gl_100':'glove-wiki-gigaword-100',
           'w2v_300':'word2vec-google-news-300'}

tk_model = 'gl_300'

if os.path.exists(f"./wv_models/{tk_dict[tk_model]}.kv"):
    word_vector = KeyedVectors.load(f"./wv_models/{tk_dict[tk_model]}.kv")
    vec_dim = len(word_vector['apple'])
    print(tk_model,vec_dim)
else:
    # load word_vector model
    word_vector = api.load(tk_dict[tk_model])
    # Save the model for future use
    word_vector.save(f"./wv_models/{tk_dict[tk_model]}.kv")
    vec_dim = len(word_vector['apple'])
    print(tk_model,vec_dim)

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])
gl_300 300


In [17]:
out = utl.load_data_format(train_dataset, 200, word_vector, 3, device, 300, tk_model)

In [18]:
a=[]
for sen, label in train_dataset:
    a.append(label)
b=torch.tensor(a).to(device)
b

tensor([5, 5, 5,  ..., 4, 5, 4], device='cuda:0')

In [19]:
out[-1]

tensor([5, 5, 5,  ..., 4, 5, 4], device='cuda:0')

In [20]:
for i in range(len(train_dataset)):
    print(i, out[1][i], b[i], train_dataset[i][1])

0 tensor(5, device='cuda:0') tensor(5, device='cuda:0') 5
1 tensor(5, device='cuda:0') tensor(5, device='cuda:0') 5
2 tensor(5, device='cuda:0') tensor(5, device='cuda:0') 5
3 tensor(5, device='cuda:0') tensor(5, device='cuda:0') 5
4 tensor(5, device='cuda:0') tensor(5, device='cuda:0') 5
5 tensor(4, device='cuda:0') tensor(4, device='cuda:0') 4
6 tensor(5, device='cuda:0') tensor(5, device='cuda:0') 5
7 tensor(5, device='cuda:0') tensor(5, device='cuda:0') 5
8 tensor(5, device='cuda:0') tensor(5, device='cuda:0') 5
9 tensor(5, device='cuda:0') tensor(5, device='cuda:0') 5
10 tensor(5, device='cuda:0') tensor(5, device='cuda:0') 5
11 tensor(5, device='cuda:0') tensor(5, device='cuda:0') 5
12 tensor(4, device='cuda:0') tensor(4, device='cuda:0') 4
13 tensor(5, device='cuda:0') tensor(5, device='cuda:0') 5
14 tensor(5, device='cuda:0') tensor(5, device='cuda:0') 5
15 tensor(5, device='cuda:0') tensor(5, device='cuda:0') 5
16 tensor(5, device='cuda:0') tensor(5, device='cuda:0') 5
17 tens

In [21]:
# # small neural network
# class lo(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.filters = nn.ModuleList([nn.Conv2d(1,128, (300, c), ) for c in [3,4,5]])
#         self.l = nn.Linear(128*3,5)
#         self.loss = nn.CrossEntropyLoss()

#     def forward(self, inp, label):
#         inputs, labels = inp
#         # out = F.tanh(self.filters[0](inp)).squeeze()
#         out = [F.tanh(fil(inputs)) for fil in self.filters]
#         print('tan',out[0].size())
#         out = [fo.squeeze() for fo in out]
#         print('sq',out[0].size())
#         out = [F.max_pool1d(fo, fo.size(2)) for fo in out]
#         print('Mp1d',out[0].size())
#         out = [fo.squeeze() for fo in out]
#         print('sq',out[0].size())
#         out = torch.cat(out, dim=1)
#         print('cat',out.size())
#         out = self.l(out.squeeze())
#         print(out.size(), labels.size())
#         # out = self.loss(out, labels-1)
#         # print('k',out)
#         return out

# low = lo().to(device)
# # # optimizer = optim.Adam(low.parameters(), lr=0.001, weight_decay=3)
# # inp = torch.rand((1,125,6))
# print(out[0].size())
# a = low(out, 1)
# print(a.size())
# print(out[1].size())
# # input = torch.randn(2, 6, 5)
# # # m = nn.MaxPool1d(input.size(2), stride=1)
# # output = F.max_pool1d(input, input.size(2)).squeeze()
# # # print(input.size())
# # # print(output.size())
# # # # print(input)
# # print(output)
# print(out[1])
# print(a)
# print(torch.sum(torch.argmax(a,dim=1)==out[1]).item())
# loss = nn.CrossEntropyLoss()
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.empty(3, dtype=torch.long).random_(5)
# # print(input.size(), target.size())
# # output = loss(input, target)

# **CNN Parameters**

**Model CNN**

In [22]:
import models.CNN as cnn
context_win = [4,5,6,7,8]
No_CNN_filter = 100               # previous models 2345-128
lr = 0.001
num_epochs = 7

# for skewed train set
if dataset_name=='sst2':
    weight=0
else:
    a = np.zeros(5)
    for d in train_dataset:
        a[d[1]-1]+=1
    weights = len(train_dataset)/(a*len(a))

In [ ]:
Cnn_model = cnn.CNN_Classifier_network(vec_dim, No_CNN_filter, out_dim, weight=weight,context_win=context_win).to(device)
# load model
e=0
ac=81
type_='best'
if os.path.exists(f"model_CNN_{type_}({utl.checkpoint_name(e,tk_model,ac,context_win)})_f_{No_CNN_filter}_{dataset_name}.pth"):
    print('loading pretrained model...')
    Cnn_model.load(f'{type_}({utl.checkpoint_name(e,tk_model,ac,context_win)})_f_{No_CNN_filter}_{dataset_name}')
    print('done')
else:
    print('No pretrained model')

In [ ]:
cnn.train(train_dataset, val_dataset, word_vector, 
          Cnn_model, context_win, num_epochs, lr, vec_dim, tk_model, dataset_name)

In [8]:
context_win = [3,4,5]
No_CNN_filter = 100               # previous models 2345-128
Cnn_model = cnn.CNN_Classifier_network(vec_dim, No_CNN_filter, out_dim, weight=weight,context_win=context_win).to(device)
# load model
e=0
ac=83
type_='best'
if os.path.exists(f"model_CNN_{type_}({utl.checkpoint_name(e,tk_model,ac,context_win)})_f_{No_CNN_filter}_{dataset_name}.pth"):
    print('loading pretrained model...')
    Cnn_model.load(f'{type_}({utl.checkpoint_name(e,tk_model,ac,context_win)})_f_{No_CNN_filter}_{dataset_name}')
    print('done')
else:
    print('No pretrained model')
test_loss, test_acc = cnn.evaluate(val_dataset, word_vector, vec_dim, tk_model, Cnn_model, max(context_win))


weights =  tensor([1., 1.], device='cuda:0')
loading pretrained model...
done
_____________________
class label: 0
.	F	T
N	81	366
P	77	346
Recall:	 0.8103044629096985
Precision:	 0.817966878414154
F1-Score: 	 1.3256044387817383
_____________________
class label: 1
.	F	T
N	77	346
P	81	366
Recall:	 0.8261851072311401
Precision:	 0.818791925907135
F1-Score: 	 1.3529473543167114
Test: loss: 0.4942| accuracy: (overall)81.8391, (non-dominant class)0.0%


In [15]:
Cnn_model = cnn.CNN_Classifier_network(vec_dim, No_CNN_filter, context_win).to(device)
Cnn_model.load(f'best(e_5_V_100_VA_79_CW_2_3_4_5)_sst2')
test_loss, test_acc = cnn.evaluate(test_dataset, word_vector, vec_dim, Cnn_model, max(context_win))
print(f'Validation: loss: {test_loss:.4f}| accuracy: {test_acc*100:.4f}')

Validation: loss: 0.8596| accuracy: 78.7679


# **Linear Parameters**

**Linear Model**

In [71]:

def predict(labels, predictions):
    predictions[predictions>=0.5] = 1
    predictions[predictions<0.5] = 0
    return torch.sum(predictions==labels)

def evaluate(validation, model):
    loss, num_pred = model(validation)
    acc = num_pred/len(validation)*100
    return loss, acc

def avg_sentence_vec(batch, word_vector):
    data ,label = [], []
    for x in batch:
        avg = 0
        count = 0
        for token in x[0]:
            if token in word_vector:
                count+=1
                avg = avg + torch.tensor(word_vector[token], dtype=torch.float32).unsqueeze(0)
        data.append(avg/count)
        label.append(torch.tensor([[int(x[1])]], dtype=torch.float32))
    return torch.concat(data, dim=0).to(device), torch.concat(label, dim=0).to(device)

class linear_Classifier_network(nn.Module):
    def __init__(self, vector_dim, out_dim) -> None:
        super().__init__()
        h1=500
        h2=256
        h3=64
        h4=out_dim
        self.classifier_network = nn.Sequential(
            nn.Linear(vector_dim, h1),
            nn.ReLU(),
            nn.Linear(h1, h2),
            nn.ReLU(),
            nn.Linear(h2, h3),
            nn.ReLU(),
            nn.Linear(h3, h4)
        )
        self.loss = nn.BCELoss()
    
    def forward(self, batch):
        inputs, labels = avg_sentence_vec(batch, word_vector)
        out = self.classifier_network(inputs)
        # print(out.shape)
        l = self.loss(out, labels)
        out_pred = out.clone().detach().to(device)
        correct = predict(labels, out_pred)
        return l, correct
    
    def save(self, ds):
        torch.save(self.state_dict(), './model_linear_'+ds+'.pth')

    def load(self, ds):
        self.load_state_dict(torch.load('./model_linear_'+ds+'.pth',
                                        map_location=device,
                                        weights_only=True))
        # self.load_state_dict(torch.load('./weights.pt'))
        return self

In [72]:
def evaluate(val_set, model):
    losses = 0
    c=0
    batch_size=100
    for i in range(0, len(val_set), batch_size):
        loss, pred_num = model(val_set[i:i+batch_size])
        losses+=(loss)
        c+=pred_num
    return (losses)/len(val_set), c/len(val_set)

In [68]:
context_win = []
lr = 0.005
num_epochs = 70
batch_size = 150

In [69]:
lin_model = linear_Classifier_network(vec_dim, out_dim).to(device)
optimizer = optim.Adam(lin_model.parameters(), lr=lr)

In [70]:
# # Training loop
# if os.path.exists("model_lin_sst2.pth"):
#     print('loading pretrained model...')
#     lin_model.load('sst2')

lin_model.train()
iters=0
max_val_acc = float('-inf')
optimizer = optim.Adam(lin_model.parameters(), lr=lr)
for epoch in range(num_epochs):
    losses = []
    correct_pred = 0
    for i in range(0, len(train_dataset), batch_size):
        iters+=1
        optimizer.zero_grad()
        loss, pred_num = lin_model(train_dataset[i:i+batch_size])
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        correct_pred+=pred_num
        # if iters%100 == 0:
        #     print(f'iteration: {i+1}| running loss={np.mean(losses):.4f}|, running acc = {correct_pred/(i+1)*100:.4f}')# validation step
        # validation step
        if (iters+1)%5 == 0:
            print('validation step...')
            val_loss, val_acc = evaluate(val_dataset, lin_model)
            if val_acc*100>max_val_acc:
                max_val_acc = val_acc*100
                print(f"saving best model, val loss = {val_loss:.4f}")
                lin_model.save(f'best({utl.checkpoint_name(epoch+1,vec_dim,int(val_acc*100))})_f_{No_CNN_filter}_sst2')
            print(f'Validation: loss: {val_loss:.4f}| accuracy: {val_acc*100:.4f}')
    lin_model.save('sst2')
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {np.mean(losses):.4f}')
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {np.mean(losses):.4f}')


../aten/src/ATen/native/cuda/Loss.cu:95: operator(): block: [0,0,0], thread: [32,0,0] Assertion `target_val >= zero && target_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:95: operator(): block: [0,0,0], thread: [33,0,0] Assertion `target_val >= zero && target_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:95: operator(): block: [0,0,0], thread: [34,0,0] Assertion `target_val >= zero && target_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:95: operator(): block: [0,0,0], thread: [35,0,0] Assertion `target_val >= zero && target_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:95: operator(): block: [0,0,0], thread: [36,0,0] Assertion `target_val >= zero && target_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:95: operator(): block: [0,0,0], thread: [37,0,0] Assertion `target_val >= zero && target_val <= one` failed.
../aten/src/ATen/native/cuda/Loss.cu:95: operator(): block: [0,0,0], thread: [39,0,0] Assertion `target_val >= zero && targe

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [18]:
lin_model = linear_Classifier_network(vec_dim).to(device)
lin_model.load(f'best(e_2_V_100_VA_78)_sst2')
test_loss, test_acc = evaluate(test_dataset, lin_model)
print(f'test: loss: {test_loss:.4f}| accuracy: {test_acc*100:.4f}')

test: loss: 0.0055| accuracy: 75.6727


In [ ]:
class CNN(nn.Module):
    def __init__(self, vector_dim, layers, context_win=3) -> None:
        super().__init__()
        self.layers = layers
        self.filters = nn.ModuleList([nn.Linear(vector_dim*context_win,1,bias=False) for _ in range(layers)])

    def forward(self, in_list):
        out = []
        for filter in self.filters:
            res = []
            for window in in_list:
                res.extend(filter(window))
            res = torch.tensor(res).unsqueeze(0)
            out.append(torch.max(res,dim=1)[0])
        out = torch.concat(out, dim=0) ######### creating trensor ##########
        return out.unsqueeze(0).to(device)

 
class CNN_Classifier_network(nn.Module):
    def __init__(self, vector_dim, layers, context_win=3) -> None:
        super().__init__()
        self.inputCNN = CNN(vector_dim, layers, context_win)
        h1=64
        h2=32
        h3=16
        h4=1
        self.classifier_network = nn.Sequential(
            nn.Linear(layers, h1),
            nn.ReLU(),
            nn.Linear(h1, h2),
            nn.ReLU(),
            nn.Linear(h2, h3),
            nn.ReLU(),
            nn.Linear(h3, h4),
            nn.ReLU()
        )
        self.loss = nn.BCELoss()
    
    def forward(self, batch):
        out=[]
        labels=[]
        for x in batch:
            input,label = load_data_format(x, word_vector)
            out.append(self.inputCNN.forward(input))    # dim = [context_win*vector_dim]*sentence_length
            labels.append(label)
        labels = torch.concat(labels, dim=0).to(device)
        out = torch.concat(out,dim=0).to(device)
        # print(out.shape, labels.shape)
        out = self.classifier_network(out)
        # print(out.shape)
        out = self.loss(out, labels)
        return out
    
    def save(self, ds):
        torch.save(self.state_dict(), './model_CNN_'+ds+'.pth')

    def load(self, ds):
        self.load_state_dict(torch.load('./model_CNN_'+ds+'.pth', 
                                        map_location=device,
                                        weights_only=True))
        # self.load_state_dict(torch.load('./weights.pt'))
        return self

In [ ]:
Cnn_model = CNN_Classifier_network(vec_dim, No_CNN_filter, context_win).to(device)
optimizer = optim.Adam(Cnn_model.parameters(), lr=lr)

In [ ]:
# Training loop

Cnn_model.train()
for epoch in range(num_epochs):
    losses = []
    count = 0
    for i in range(0,len(train_dataset),batch_size):
        count+=1
        optimizer.zero_grad()
        loss = Cnn_model(train_dataset[i:i+batch_size])
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        print(f'iteration: {count}| running loss={np.mean(losses)}| batch: {i+1}:{i+batch_size}')
    Cnn_model.save('sst2')
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
# Training loop


iteration: 1| running loss=1.2860336303710938| batch: 1:150
iteration: 2| running loss=1.2707149386405945| batch: 151:300
iteration: 3| running loss=1.1892604033152263| batch: 301:450
iteration: 4| running loss=1.1386961489915848| batch: 451:600
iteration: 5| running loss=1.1431633114814759| batch: 601:750


In [45]:
out = []
a = torch.tensor([1,2,3])
out.extend([torch.max(a)])

In [24]:
a = torch.ones(30)
b = torch.ones(30)
torch.cat([a,b], )

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1.])

# rough


In [9]:
from models.LSTM import LSTMModel
import torch

l = LSTMModel(300, 300, 1)

out = l(torch.randn(2,12,300))

In [10]:
out[1][0].shape

torch.Size([2, 2, 300])

In [11]:
out[0].shape

torch.Size([2, 12, 300])